In [130]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [131]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')
gpt_model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [132]:
input_ids = np.array([tokenizer.encode('이때')])
# print(type(input_ids))
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_k=20)
sentence = tokenizer.decode(output[0].numpy().tolist())
print(sentence)

이때만 해도 그 정도 수준은 아니다.
오히려, 그 수준이라는 건, 한국도 비슷한 수준이 아닌가 하는 생각이 들 정도.</d> 게임 내에선 상당히 많은 게임들 중에서, '스타트렉'이라는 이름의 몬스터를 생성할 수 있다.
맵에 위치한 맵에서 스타트렉을 처치하면 몬스터, 즉 아이템을 얻을 수 있는데, 여기에 더해 '스타트렉'이라는 이름이 더해졌다.
맵이 너무 좁은데다가 캐릭터들 간의 밸런스 조정이 필요해서, '


In [133]:
input_ids = np.array([tokenizer.encode('이때')])
# print(type(input_ids))
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_p=0.95)
sentence = tokenizer.decode(output[0].numpy().tolist())
print(sentence)

이때 전 세계에서 몰려들어가면 한국에서도 일본이나 베트남과는 달리 '한국산' 반송되는 경우가 많아 반송되는 경우가 많다.
또한 베트남에 한국산 반송 여부가 확인되는 사례가 거의 없기 때문에 한국산을 사는 것에 대해 부담감이 적지 않다.</d> AAB는 미국의 프로듀서이자 디자이너, 작곡가, 엔지니어가 함께 일하는 회사이다.
1980년대 초에 이 회사의 CEO가 되었다.
또한 AAB는 2017년에 사장이 되었다.
AB


In [134]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]
print(sents[0:2])
print(len(sents))

['그때에 김첨지는 대수롭지 않은듯이,', '만일 김첨지가 주기를 띠지 않았던들 한 발을 대문에 들여놓았을 제 그곳을 지배하는 무시무시한 정적(靜寂) ― 폭풍우가 지나간 뒤의 바다 같은 정적이 다리가 떨렸으리라.']
284


In [135]:
def get_data():
  for sent in sents:
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode(sent ) 
    yield bos_token + sent + eos_token

In [136]:
dataset = tf.data.Dataset.from_generator(get_data, output_types=tf.int32)
dataset = dataset.padded_batch(batch_size=8, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [137]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    0 12858  8022  9324  8364  9272 20897 33336 10091 23272     1     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    0 19089  9324  8364  9737 42414 10481  8263 18318  7285  9036 19846
   9026 10924 15994  7128 23928  9037 35433 21299 15218  7556 27232  9040
  11360  6433  2010   384   739   679 39798 15576 50866 16358 11099  9239
   9040 10018 23948  9940  7422 17082 25462     1     3     3     3     3
      3     3     3     3]
 [    0 30181  9337  9164 13530 41732  9080  9548  9784  8139  7513  9989
   9079 28936   739  6976 22506 27533  7763  8463  7235 24257  8420 10171
   7182 45006  9208  6889  8689  9796  9056  7791  9129  7395  9366  9818
   7198  7321  9174 20579 19897 11520  9676  9220  8006  8528  8137  9036
  15059  8367 18005     1]
 [    0  9676  9220 

In [138]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [139]:
steps = len(sents) // 8 + 1
print(steps)

36


In [140]:
import tqdm
EPOCHS = 10

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = gpt_model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, gpt_model.trainable_variables)
      adam.apply_gradients(zip(grads, gpt_model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.90390086


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.60405207


  0%|          | 0/36 [00:00<?, ?it/s]

...

[Epoch:    8] cost = 0.251193196


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    9] cost = 0.232570022


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:   10] cost = 0.222360566


In [144]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.save_weights(save_path)

In [142]:
input_ids = np.array([tokenizer.encode('이때')])
output = gpt_model.generate(input_ids, max_length=100)
sentence = tokenizer.decode(output[0].numpy().tolist(),skip_special_tokens=True)
print(sentence)

이때 “에미를 붙을 이 오라질 놈들 같으니, 이놈 내가 돈이 없을 줄 알고.”


In [150]:
input_ids = np.array([tokenizer.encode('이때')])
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_p=0.95)
sentence = tokenizer.decode(output[0].numpy().tolist(),skip_special_tokens=True)
print(sentence)

이때 이 말이 저도 모를 사이에 불쑥 김첨지의 입에서 떨어졌다.
